In [15]:
import pandas as pd            
import numpy as np             
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
%matplotlib inline

In [110]:
# Read csv
df = pd.read_csv('CrowdstormingDataJuly1st.csv')
# Drop meaningless columns, we don't need the whole player name nor the birthday
df.drop(['player','birthday'], 1, inplace=True)
# Drop dyads where there are no photoID
df.dropna(axis=0, subset=['photoID'], inplace=True)
df.drop('photoID', 1, inplace=True)
# Drop dyads where there are no rating
df.dropna(axis=0, how='all', subset=['rater1', 'rater2'], inplace=True)

In [111]:
# First, try without 'racist data'
df.drop(['refNum','refCountry','Alpha_3','meanIAT','nIAT','seIAT','meanExp','nExp','seExp'], 1, inplace=True)

In [180]:
# Need to sum for the aggregation
sumAggIndexes = ['playerShort', 'games', 'victories', 'ties', 'defeats', 'goals', 'yellowCards','yellowReds', 'redCards']
dfSumAgg = df[sumAggIndexes]
aggregatedWithSum = dfSumAgg.groupby('playerShort').sum()

# No sum for aggregation
dfIdentityAgg = df[['playerShort', 'club', 'leagueCountry', 'height', 'weight',
       'position', 'rater1', 'rater2']]
identity = lambda x: x.iloc[0]
aggregatedWithIdentity = dfIdentityAgg.groupby('playerShort').agg(identity)

# Concatenate
dfd = pd.concat([aggregatedWithIdentity, aggregatedWithSum], axis=1)

In [192]:
# skin color
colors = np.array([0, 0.25, 0.5, 0.75, 1])
def find_nearest(array,value):
    idx = (np.abs(array-value)).argmin()
    return array[idx]

meanColor = (dfd['rater1']+dfd['rater2'])/2
fixedColor = meanColor.apply(lambda x: find_nearest(colors, x))

# TODO: concat fixedColor


TypeError: Can only append a Series if ignore_index=True or if the Series has a name

In [13]:
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.3, random_state=0)

0                 lucas-wilchez
1                    john-utaka
2                   abdon-prats
3                    pablo-mari
4                    ruben-pena
5                  aaron-hughes
6            aleksandar-kolarov
7              alexander-tettey
8             anders-lindegaard
9                  andreas-beck
10             antonio-rukavina
11               ashkan-dejagah
12            benedikt-hoewedes
13                  chris-baird
14                  chris-brunt
15               daniel-schwaab
16                  dennis-aogo
17             george-mccartney
18             gylfi-sigurdsson
19               ivan-obradovic
20                  jan-moravek
21                jan-rosenthal
22                  jonny-evans
23        kyriakos-papadopoulos
24                  marko-marin
25                 mats-hummels
26                  mesut-oezil
27              milorad-pekovic
28                nemanja-vidic
29                neven-subotic
                  ...          
145998  

In [161]:
df.columns.get_values()

array(['playerShort', 'club', 'leagueCountry', 'height', 'weight',
       'position', 'games', 'victories', 'ties', 'defeats', 'goals',
       'yellowCards', 'yellowReds', 'redCards', 'rater1', 'rater2'], dtype=object)

In [57]:
d = pd.DataFrame({'A' : ['foo', 'bar', 'foo', 'bar',
                  'foo', 'bar', 'foo', 'foo'],
                  'B' : ['one', 'one', 'two', 'three',
                          'two', 'two', 'one', 'three'],
                  'C' : np.random.randn(8),
                  'D' : np.random.randn(8),
                  'E' : ['1','0','1','0','1','0','1','1']})

In [63]:
g = d[['A','E']].groupby('A')
g.groups

{'bar': Int64Index([1, 3, 5], dtype='int64'),
 'foo': Int64Index([0, 2, 4, 6, 7], dtype='int64')}